In [1]:
import tensorflow as tf

In [2]:
import dataset

In [3]:
root_dir = '/Volumes/Transcend/Data/Steel/'
csv_file = root_dir + 'train.csv'
train_dir = root_dir + 'train/'
mask_dir = root_dir + 'mask/'

In [4]:
# img_dataset = dataset.gen_dataset()

In [5]:
img_dataset = tf.data.Dataset.from_generator(dataset.gen_dataset, (tf.float32, tf.float32), (tf.TensorShape([256, 1600, 3]), tf.TensorShape([256, 1600, 4])))

In [6]:
img_dataset = img_dataset.repeat()
img_dataset = img_dataset.batch(2)

In [7]:
import importlib

In [8]:
import BaseUNet

In [9]:
importlib.reload(BaseUNet)

<module 'BaseUNet' from '/Users/sri/P/Steel/BaseUNet.py'>

In [10]:
model = BaseUNet.UNet()

In [11]:
loss_obj = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_iou = tf.keras.metrics.MeanIoU(num_classes=4)

In [13]:
@tf.function
def train(dataset, loss_obj, train_loss, train_metric, model, optimizer):
    count = 0
    for image, masks in dataset:
        count += 1
        if count > 10:
            break
        with tf.GradientTape() as tape:
            predictions = model(image)
            loss = loss_obj(masks, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        train_loss(loss)
        train_metric(masks, predictions)

In [14]:
train(img_dataset, loss_obj, train_loss, train_iou, model, optimizer)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100%|██████████| 50272/50272 [00:11<00:00, 4446.46it/s]


InvalidArgumentError:   assertion failed: [`labels` out of bound] [Condition x < y did not hold element-wise:] [x (confusion_matrix/control_dependency:0) = ] [0 0 0...] [y (confusion_matrix/Cast_2:0) = ] [4]
	 [[{{node cond/then/_0/cond_1/then/_454/confusion_matrix/assert_less/Assert/AssertGuard/else/_919/Assert}}]]
	 [[ReduceDataset]] [Op:__inference_train_4822]

Function call stack:
train -> tf_data_experimental_scan_scan_body


In [ ]:
assert False

In [15]:
for image, label_image in img_dataset:
    out = model(image)
    print(out.shape)
    assert False

100%|██████████| 50272/50272 [00:10<00:00, 4634.79it/s]


(2, 256, 1600, 4)


AssertionError: 